# Imports

In [1]:
from PIL import Image
import numpy as np
import copy
import cv2 as cv
import os
from scipy.sparse import csr_matrix, hstack
import matplotlib.pyplot as plt
import seaborn as seab
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from sklearn import metrics

In [2]:
all_data = pd.read_csv("../Dataset/PS_20174392719_1491204439457_log.csv")

In [64]:
print len(all_data)

6362620


In [65]:
print all_data.keys()
all_data.shape

Index([u'step', u'type', u'amount', u'nameOrig', u'oldbalanceOrg',
       u'newbalanceOrig', u'nameDest', u'oldbalanceDest', u'newbalanceDest',
       u'isFraud', u'isFlaggedFraud'],
      dtype='object')


(6362620, 11)

In [10]:
for i in range(len(all_data.isFraud)):
    if all_data.isFlaggedFraud[i] == 1 and all_data.isFraud[i] !=1:
        print i

In [ ]:
inds = []
indsn = []
for i in range(len(all_data.isFraud)):
    if all_data.isFraud == 1:
        inds.append()

In [63]:
print all_data.iloc[0,:]
all_data.loc[0:10,['step','amount']]

step                        1
type                  PAYMENT
amount                9839.64
nameOrig          C1231006815
oldbalanceOrg          170136
newbalanceOrig         160296
nameDest          M1979787155
oldbalanceDest              0
newbalanceDest              0
isFraud                     0
isFlaggedFraud              0
Name: 0, dtype: object


,step,amount
0,1,9839.64
1,1,1864.28
2,1,181.00
3,1,181.00
4,1,11668.14
5,1,7817.71
6,1,7107.77
7,1,7861.64
8,1,4024.36
9,1,5337.77


# remove feature isFlagged fraud

In [49]:
new_all_data = pd.DataFrame()
new_all_data = new_all_data.append(all_data.loc[:,['step','type','amount','nameOrig','oldbalanceOrg','newbalanceOrig','nameDest','oldbalanceDest','newbalanceDest','isFraud']])

In [50]:
print len(new_all_data)

6362620


In [73]:
set(new_all_data.type)

{'CASH_IN', 'CASH_OUT', 'DEBIT', 'PAYMENT', 'TRANSFER'}

In [74]:
# new_filterdata = pd.DataFrame()
# for i in range(len(new_all_data.type)):
#     if(new_all_data.type[i] =='CASH_OUT'or new_all_data.type[i] =='TRANSFER'):
#         new_filterdata = new_filterdata.append(new_all_data.iloc[i,:])
#         print len(new_filterdata)

In [76]:
# print len(new_filtereddata)

In [77]:
new_filterdata = pd.DataFrame()

In [78]:
new_filterdata= new_filterdata.append(new_all_data.loc[new_all_data.loc[:,'type'] == 'TRANSFER',:])
len(new_filterdata)

532909

In [79]:
new_filterdata= new_filterdata.append(new_all_data.loc[new_all_data.loc[:,'type'] == 'CASH_OUT',:])
len(new_filterdata)

2770409

In [80]:
print new_filterdata.keys()

Index([u'step', u'type', u'amount', u'nameOrig', u'oldbalanceOrg',
       u'newbalanceOrig', u'nameDest', u'oldbalanceDest', u'newbalanceDest',
       u'isFraud'],
      dtype='object')


In [109]:
new_filterdata = pd.DataFrame()
new_filterdata= new_filterdata.append(new_all_data.loc[new_all_data.loc[:,'type'] == 'TRANSFER',:],ignore_index=True)
len(new_filterdata)
new_filterdata= new_filterdata.append(new_all_data.loc[new_all_data.loc[:,'type'] == 'CASH_OUT',:],ignore_index=True)
len(new_filterdata)
print new_filterdata.keys()

Index([u'step', u'type', u'amount', u'nameOrig', u'oldbalanceOrg',
       u'newbalanceOrig', u'nameDest', u'oldbalanceDest', u'newbalanceDest',
       u'isFraud'],
      dtype='object')


In [110]:
# for i in range(len(new_filterdata.type)):
#     if(new_filterdata.type[i] == 'TRANSFER'):
#         new_filterdata.type[i] = 1
#     else:
#         new_filterdata.type[i] = 2

In [111]:
print set(new_filterdata.type)

set(['CASH_OUT', 'TRANSFER'])


In [112]:
new_filterdata.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
0,1,TRANSFER,181.00,C1305486145,181.00,0.0,C553264065,0.00,0.00,1
1,1,TRANSFER,215310.30,C1670993182,705.00,0.0,C1100439041,22425.00,0.00,0
2,1,TRANSFER,311685.89,C1984094095,10835.00,0.0,C932583850,6267.00,2719172.89,0
3,1,TRANSFER,62610.80,C1976401987,79114.00,16503.2,C1937962514,517.00,8383.29,0
4,1,TRANSFER,42712.39,C283039401,10363.39,0.0,C1330106945,57901.66,24044.18,0


In [120]:
copydata = copy.deepcopy(new_filterdata)

In [121]:
copydata.tail()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
2770404,742,CASH_OUT,63416.99,C994950684,63416.99,0.0,C1662241365,276433.18,339850.17,1
2770405,743,CASH_OUT,1258818.82,C1436118706,1258818.82,0.0,C1240760502,503464.50,1762283.33,1
2770406,743,CASH_OUT,339682.13,C786484425,339682.13,0.0,C776919290,0.00,339682.13,1
2770407,743,CASH_OUT,6311409.28,C1162922333,6311409.28,0.0,C1365125890,68488.84,6379898.11,1
2770408,743,CASH_OUT,850002.52,C1280323807,850002.52,0.0,C873221189,6510099.11,7360101.63,1


In [122]:
copydata.head()


,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
0,1,TRANSFER,181.00,C1305486145,181.00,0.0,C553264065,0.00,0.00,1
1,1,TRANSFER,215310.30,C1670993182,705.00,0.0,C1100439041,22425.00,0.00,0
2,1,TRANSFER,311685.89,C1984094095,10835.00,0.0,C932583850,6267.00,2719172.89,0
3,1,TRANSFER,62610.80,C1976401987,79114.00,16503.2,C1937962514,517.00,8383.29,0
4,1,TRANSFER,42712.39,C283039401,10363.39,0.0,C1330106945,57901.66,24044.18,0


In [103]:
# copydata.reset_index()

In [123]:
copydata.loc[copydata.type == 'TRANSFER', 'type'] = 1
copydata.loc[copydata.type == 'CASH_OUT', 'type'] = 2

In [124]:
copydata.tail()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
2770404,742,2,63416.99,C994950684,63416.99,0.0,C1662241365,276433.18,339850.17,1
2770405,743,2,1258818.82,C1436118706,1258818.82,0.0,C1240760502,503464.50,1762283.33,1
2770406,743,2,339682.13,C786484425,339682.13,0.0,C776919290,0.00,339682.13,1
2770407,743,2,6311409.28,C1162922333,6311409.28,0.0,C1365125890,68488.84,6379898.11,1
2770408,743,2,850002.52,C1280323807,850002.52,0.0,C873221189,6510099.11,7360101.63,1


In [143]:
print len(set(copydata.nameOrig))
print len(set(copydata.nameDest))

2768630
509565


In [125]:
copydata.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
0,1,1,181.00,C1305486145,181.00,0.0,C553264065,0.00,0.00,1
1,1,1,215310.30,C1670993182,705.00,0.0,C1100439041,22425.00,0.00,0
2,1,1,311685.89,C1984094095,10835.00,0.0,C932583850,6267.00,2719172.89,0
3,1,1,62610.80,C1976401987,79114.00,16503.2,C1937962514,517.00,8383.29,0
4,1,1,42712.39,C283039401,10363.39,0.0,C1330106945,57901.66,24044.18,0


In [ ]:
train_data = copydata.loc[:,['step','type','amount','nameOrig','oldbalanceOrg','newbalanceOrig','nameDest','oldbalanceDest','newbalanceDest','isFraud']]

In [139]:
data = np.array(copydata)
train_data = data[:,0:len(data[0]) - 1]
train_labels = data[:,-1]

In [140]:
print train_data[2770407]
print train_labels[0]
print train_labels[1]

[743 2 6311409.28 'C1162922333' 6311409.28 0.0 'C1365125890' 68488.84
 6379898.11]
1
0


In [128]:
from sklearn.svm import SVC

In [ ]:
# clf = SVC(gamma='auto')
# clf.fit(train_data,train_labels)

In [145]:
unique_nameOrig = set(copydata.nameOrig)
unique_nameDest = set(copydata.nameDest)


In [146]:
copy2 = copy.deepcopy(copydata)

In [ ]:
index = 1
for val in unique_nameOrig:
    copydata.loc[copydata.nameOrig == val, 'nameOrig'] = index
    index += 1
print "name orig completed"
index = 1
for val in unique_nameDest:
    copydata.loc[copydata.nameDest == val, 'nameDest'] = index
    index += 1
print "name dest completed"

In [143]:
print len(set(copydata.nameOrig))
print len(set(copydata.nameDest))

2768630
509565
